### This jupyter notebook contains the algorithmic part of the proof that the _Ruperthedron_ is Rupert but not locally Rupert.

This document consists of two parts:
    
1) [**RUP is Rupert**](#2): We prove that the _Ruperthedron_ is Rupert with Nieuwland number at least 1.003.
2) [**RUP is not locally Rupert**](#3): We show that the _Ruperthedron_ is not locally Rupert. Following the outline of the paper, the proof is split into two steps:
   - Check that each $B_{i,j}$ indeed lies inside one $C_k$.
   - Check that the rational local theorem can be applied for each $C_k$. 

In [ ]:
from helper_functions import * # import all kinds of useful helper functions
from ruperthedron import RUP # import the Noperthedron as poly 

import pandas as pd # to read the tabular solution tree

from tqdm import tqdm

INTERVAL_DENOMINATOR = 10000
R_DENOMINATOR = 1000
omega=6/10000

kappa=10**(-10)

C=pd.read_csv("../data/Ck.csv", sep=',')
MAPPING=pd.read_csv("../data/mapping.csv", sep=',')

RatRUP=matrix(QQ,90,3) # define the rational approximation of the Ruperthedron
for row in range(RUP.nrows()):
    for col in range(RUP.ncols()):
        RatRUP[row,col]=floor(RUP[row,col]*10**16)/10**16

In [ ]:
print("C looks like:")
C.head()

In [ ]:
print("MAPPING looks like:")
MAPPING.head()

# The Ruperthedron is Rupert

In [ ]:
# Solution from the paper
theta1    =   29/100
phi1      =   29/100
theta2    =    2/100
phi2      =  227/100
alpha     = -102/100
nieuwland = 1003/1000

P=RUP*M_exact(theta1,phi1).T*R_exact(alpha).T*nieuwland
Q=RUP*M_exact(theta2,phi2).T

# Rational approximations of solution for faster computation
P_approx = RatRUP*M(theta1,phi1).T*R(alpha).T*nieuwland
Q_approx = RatRUP*M(theta2,phi2).T

In [ ]:
def inequalityCheck(X,A,B,C):
    #Input: X,A,B,C points in R^2
    #If the function returns true then X is inside ABC. Note: converse not necessarily correct. 
    
    # Uses the fact that D is inside ABC iff (0,0) is inside the triange A-D,B-D,C-D; and Lemma 26
    
    AA=A-X
    BB=B-X
    CC=C-X

    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2), i.e. rotation 90 degrees counter-clockwise
    if ScalarProduct_exact(R_pi_2*AA,BB)<=0:
        return False
    if ScalarProduct_exact(R_pi_2*BB,CC)<=0:
        return False
    if ScalarProduct_exact(R_pi_2*CC,AA)<=0:
        return False
        
    return True

In [ ]:
## For each point of P, we want to find 3 points of Q, such that the point of P lies inside the triangle of Q
## To find promising three points of Q, we first use rational approximations of the P and Q.
## Then, we do the check using the exact algebraic numbers.

for i in range(90): ## iterate over the points of P, that have to be inside Q
    inside=False
    for j in range(90):
        if inside: break
        for k in range(90):
            if inside: break
            for l in range(90):
                if inside: 
                    break
                if (inequalityCheck(P_approx[i,:].T,Q_approx[j,:].T,Q_approx[k,:].T,Q_approx[l,:].T)): 
                    # to increase speed, we test first whether the rational approximation is inside  
                    # if yes we check whether the point P[i] is indeed inside a triangle of Q
                    if (inequalityCheck(P[i,:].T,Q[j,:].T,Q[k,:].T,Q[l,:].T)):
                        inside=True
                        print(f"Point P[{i}] is inside the triange Q[{j}], Q[{k}], Q[{l}]")
    assert inside, f"P[{i}] is outside of Q!"
print(f"We have proven, that RUP is Rupert with Nieuwland constant at least {nieuwland}!")

# The Ruperthedron is not locally Rupert

Here we follow the approach outlined in the paper: 
1. We have to prove, that there are no solutions in the sets $C_k\subset \mathbb{R}^5$ for $k=0,...,3534$ and
2. For each set $B_{i,j}$ there exists a set $C_k$ such that $B_{i,j} \subset C_k$.

The definition of the sets $B_{i,j}$ is explained in the paper. The sets $C_k$ are stored in the table $C$. 

In order to conclude we have to check that:
1. [**Each $B_{i,j}$ lies inside corresponding $C_k$**](#Each-$B_{i,j}$-lies-inside-corresponding-$C_k$): We check that each $B_{i,j}$ indeed lies inside one $C_k$.
2. [**Verifying the local theorem for $C_k$**](#Verifying-the–local-theorem-for-the-$C_k$): We check that the rational local theorem can be applied for each $C_k$. 

## Each $B_{i,j}$ lies inside corresponding $C_k$

In [ ]:
A=[omega*i for i in range(2700)]

for i in tqdm(range(700)):
    for j in range(2619):

        index=i*2619+j
        assert MAPPING["i"].iloc[int(index)]==i, f"There is a problem in row {index}"
        assert MAPPING["j"].iloc[int(index)]==j, f"There is a problem in row {index}"

        C_index=MAPPING["C_index"].iloc[int(index)]

        assert int(C["ThetaMin"].iloc[int(C_index)])/INTERVAL_DENOMINATOR<=A[i]-omega
        assert int(C["ThetaMax"].iloc[int(C_index)])/INTERVAL_DENOMINATOR>=A[i]+omega
        assert int(C["PhiMin"].iloc[int(C_index)])/INTERVAL_DENOMINATOR<=A[j]-omega
        assert int(C["PhiMax"].iloc[int(C_index)])/INTERVAL_DENOMINATOR>=A[j]+omega

## Verifying the local theorem for the $C_k$

In [ ]:
for row in tqdm(C.itertuples(), total=len(C)): 
    T1_min=Integer(row.ThetaMin)/INTERVAL_DENOMINATOR
    T1_max=Integer(row.ThetaMax)/INTERVAL_DENOMINATOR
    V1_min=Integer(row.PhiMin)/INTERVAL_DENOMINATOR
    V1_max=Integer(row.PhiMax)/INTERVAL_DENOMINATOR
    T2_min=Integer(row.ThetaMin)/INTERVAL_DENOMINATOR
    T2_max=Integer(row.ThetaMax)/INTERVAL_DENOMINATOR
    V2_min=Integer(row.PhiMin)/INTERVAL_DENOMINATOR
    V2_max=Integer(row.PhiMax)/INTERVAL_DENOMINATOR
    A_min =-omega
    A_max =omega

    t1=(T1_min+T1_max)/2
    v1=(V1_min+V1_max)/2
    t2=(T2_min+T2_max)/2
    v2=(V2_min+V2_max)/2
    a=(A_min+A_max)/2

    eps=max(T1_max-T1_min,V1_max-V1_min,T2_max-T2_min,V2_max-V2_min,A_max-A_min)/2
  
    P1_index=row.P1_Q1_index
    P2_index=row.P2_Q2_index
    P3_index=row.P3_Q3_index
    Q1_index=row.P1_Q1_index
    Q2_index=row.P2_Q2_index
    Q3_index=row.P3_Q3_index
  
    P1=RatRUP[P1_index,:].T
    P2=RatRUP[P2_index,:].T
    P3=RatRUP[P3_index,:].T
    Q1=RatRUP[Q1_index,:].T
    Q2=RatRUP[Q2_index,:].T
    Q3=RatRUP[Q3_index,:].T
    
    r=Integer(row.r)/R_DENOMINATOR 
    assert r>0, "r should be positive!"
    
    sigma_P=0
    sigma_Q=0
    
    R_pi_2=matrix(QQ,[[0,-1],[1,0]]) ## is set to R(pi/2)
    
    X_1=X(t1,v1)
    X_2=X(t2,v2)
    
    M_t1_v1=M(t1,v1)
    M_t2_v2=M(t2,v2)

    r1=(R(a)*M_t1_v1*P1 -M_t2_v2*Q1)
    r2=(R(a)*M_t1_v1*P2 -M_t2_v2*Q2)
    r3=(R(a)*M_t1_v1*P3 -M_t2_v2*Q3)
    
    delta=3*kappa+max(sqrt_upper(NormSquared(r1)),
                      sqrt_upper(NormSquared(r2)),
                      sqrt_upper(NormSquared(r3)))/2
    
    g=2*eps*(142/100+eps)
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (A_eps^Q): s_p condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    assert (-1)**sigma_P*ScalarProduct(X_1,P1)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_P*ScalarProduct(X_1,P2)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_P*ScalarProduct(X_1,P3)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (A_eps^Q): s_q condition ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
  
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q1)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q2)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert (-1)**sigma_Q*ScalarProduct(X_2,Q3)>142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
    ### Spanning Inequalities ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert ScalarProduct(R_pi_2*M_t1_v1*P1,M_t1_v1*P2)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t1_v1*P2,M_t1_v1*P3)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t1_v1*P3,M_t1_v1*P1)>g+6*kappa, f"Cannot apply rational local Theorem!"
    
    assert ScalarProduct(R_pi_2*M_t2_v2*Q1,M_t2_v2*Q2)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q2,M_t2_v2*Q3)>g+6*kappa, f"Cannot apply rational local Theorem!"
    assert ScalarProduct(R_pi_2*M_t2_v2*Q3,M_t2_v2*Q1)>g+6*kappa, f"Cannot apply rational local Theorem!"

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### Points are far from the origin ####
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    assert sqrt_lower(NormSquared(M_t2_v2*Q1))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert sqrt_lower(NormSquared(M_t2_v2*Q2))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    assert sqrt_lower(NormSquared(M_t2_v2*Q3))>= r+142/100*eps+3*kappa, f"Cannot apply rational local Theorem!"
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ### (B_eps^\Q) inequality ###
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    for i in range(1,4):
        if i==1:
            Qi_Index=Q1_index
            Qi=Q1
        if i==2:
            Qi_Index=Q2_index
            Qi=Q2
        if i==3:
            Qi_Index=Q3_index
            Qi=Q3
    
        denom1=sqrt_upper(NormSquared(M_t2_v2*Qi))+142/100*eps+3*kappa
    
        for A_index in range(RatRUP.nrows()):
            if A_index==Qi_Index:
                continue
                
            A=RatRUP[A_index,:].T

            nom1=ScalarProduct(M_t2_v2*Qi,M_t2_v2*Qi-M_t2_v2*A)
            nom2=10*kappa+2*eps*(sqrt_upper(NormSquared(Qi-A))+2*kappa)*(142/100+eps)
            
            nom=nom1-nom2

            denom2=sqrt_upper(NormSquared(M_t2_v2*(Qi-A)))+2*142/100*eps+6*kappa
      
            denom=denom1*denom2
      
            frac=nom/denom
      
            assert frac > (224/100*eps+delta)/r, f"Cannot apply rational local Theorem!"

print("Rational local Theorem can always be applied!")